### Fine-tuning

In [ ]:
!pip install -q datasets transformers accelerate bitsandbytes peft evaluate
!pip install -q -U bitsandbytes
!pip install --upgrade transformers
!pip install datasets evaluate rouge-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 104.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.3
    Uninstalling transformers-4.51.3:
      Successfully uninstalled transformers-4.51.3


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=678e4035acf81da2dcee84db9231d5a5c187f5d935623a4bc1b0b2cf3dc7c5a1
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [ ]:
from huggingface_hub import login
# login("hf_xxxxxxxxxxxxxxxxxxxxx") rAENAULT
login("hf_xxxxxxxxxxxxxxxxxxxxx")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tarfile

# Path to your .tar.gz file
tar_file_path = '/content/liputan6_data.tar.gz'
# tar_file_path = '/content/drive/MyDrive/ColabNotebooks/NLP_NaturalLanguageProcessing/liputan6_data.tar.gz'


# Extract the .tar.gz file
with tarfile.open(tar_file_path, 'r:gz') as tar:
    tar.extractall(path='extracted_data')

In [ ]:
import os
import json

data_dir = '/content/extracted_data/liputan6_data/canonical/train'
combined_dataset = []

for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.json'):
            with open(os.path.join(root, file), 'r', encoding='utf-8') as f:
                try:
                    data = json.load(f)
                    if 'clean_article' in data and 'clean_summary' in data:
                        article = ' '.join([' '.join(sent) for sent in data['clean_article']])
                        summary = ' '.join([' '.join(sent) for sent in data['clean_summary']])
                        combined_dataset.append({'text': article, 'summary': summary})
                except Exception as e:
                    print(f"Skipping {file}: {e}")

In [ ]:
from datasets import Dataset

# Convert to HuggingFace Dataset format
hf_dataset = Dataset.from_list(combined_dataset)
hf_dataset = hf_dataset.select(range(500))  ### Ambil sebagian dari dataset (hanya 500)
#kurangi value (500) menjadi 450 (-50) jikalau eror kar

# === Format dataset for summarization ===
def format_instruction(example):
    article = example["text"]
    summary = example["summary"]
    prompt = f"Buat ringkasan dari teks berikut:\n{article.strip()}\n"
    return {"text": prompt, "summary": summary}

# Apply the formatting function to the dataset
hf_dataset = hf_dataset.map(format_instruction, remove_columns=hf_dataset.column_names)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# === Split into train/eval sets ===
split_dataset = hf_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
import os
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
from transformers import BitsAndBytesConfig

In [ ]:
# === Load Tokenizer and Model (Quantized) ===
# model_name = "mistralai/Mistral-7B-Instruct-v0.2"
model_name = "TheBloke/vicuna-7b-1.1-HF"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.float16
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggin

...

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
# Jika error muncul saat fine-tuning, kamu bisa coba lihat nama modul yang cocok dengan:
# print([name for name, _ in model.named_modules() if "q_proj" in name or "v_proj" in name])

In [ ]:
!pip uninstall numpy
!pip install numpy==1.26.4

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Would remove:
    /usr/local/bin/f2py
    /usr/local/bin/numpy-config
    /usr/local/lib/python3.11/dist-packages/numpy-2.0.2.dist-info/*
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libgfortran-040039e1-0352e75f.so.5.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libquadmath-96973f99-934c22de.so.0.0.0
    /usr/local/lib/python3.11/dist-packages/numpy.libs/libscipy_openblas64_-99b71e71.so
    /usr/local/lib/python3.11/dist-packages/numpy/*
Proceed (Y/n)? Y
  Successfully uninstalled numpy-2.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 95.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which i

In [ ]:
import numpy as np
print(np.__version__)

1.26.4


In [ ]:
from peft import PeftModel, prepare_model_for_kbit_training, LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer

# === Prepare model for LoRA fine-tuning ===
model.gradient_checkpointing_enable()
model.enable_input_require_grads()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
)

if not isinstance(model, PeftModel) and not hasattr(model, "peft_config"):
    model = get_peft_model(model, lora_config)

model.train()

# === Tokenization ===
def tokenize(example):
    prompts = [f"[INST] Buat ringkasan dari artikel berikut:\n{text.strip()}\n[/INST]" for text in example["text"]]
    full_inputs = [prompt + " " + summary.strip() for prompt, summary in zip(prompts, example["summary"])]

    tokens = tokenizer(full_inputs, truncation=True, padding="max_length", max_length=512)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_train = train_dataset.map(tokenize, batched=True)
tokenized_eval = eval_dataset.map(tokenize, batched=True)

tokenized_train.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
tokenized_eval.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

# === Training Arguments (choose either bf16 or fallback to fp32) ===
training_args = TrainingArguments(
    output_dir="./vicuna7b-finetuned-liputan6",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    save_total_limit=2,
    bf16=True,  # ✅ Use bf16 if your GPU supports it (e.g., T4, A100)
    # fp16=False,  # ❌ Do not use fp16 with 8-bit optimizer on Colab unless you know it's safe
    gradient_checkpointing=True,
    optim="paged_adamw_8bit",  # Still using bitsandbytes optimizer
    learning_rate=2e-5,
    lr_scheduler_type="cosine",
    warmup_steps=10,
    report_to="none"
)

# === Trainer ===
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    tokenizer=tokenizer,
)

trainer.train()

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

<ipython-input-11-498ab87b705b>:58: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
10,4.106700
20,3.702400
30,3.401600
40,3.580600
50,3.686000
60,3.137200
70,3.671400
80,3.139100
90,3.553500
100,3.588000


TrainOutput(global_step=150, training_loss=3.4625538635253905, metrics={'train_runtime': 12510.7547, 'train_samples_per_second': 0.096, 'train_steps_per_second': 0.012, 'total_flos': 2.4372773388288e+16, 'train_loss': 3.4625538635253905, 'epoch': 3.0})

In [ ]:
from peft import PeftModel

# Assuming `model` is your LoRA model and has been trained
model.push_to_hub("rizal003/vicuna7b-finetuned-liputan6")
tokenizer.push_to_hub("rizal003/vicuna7b-finetuned-liputan6")

adapter_model.safetensors:   0%|          | 0.00/16.8M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rizal003/vicuna7b-finetuned-liputan6/commit/68ac59a4055db98debda62aa140c6520960203fd', commit_message='Upload tokenizer', commit_description='', oid='68ac59a4055db98debda62aa140c6520960203fd', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rizal003/vicuna7b-finetuned-liputan6', endpoint='https://huggingface.co', repo_type='model', repo_id='rizal003/vicuna7b-finetuned-liputan6'), pr_revision=None, pr_num=None)

### Evaluation

In [ ]:
# Install necessary packages if they are not already installed
!pip install -q transformers accelerate bitsandbytes evaluate

from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator
import torch
import os
from datasets import Dataset
import evaluate

# Set up an offload directory on the disk (make sure the directory exists or create it)
offload_dir = "/content/model_offload"

if not os.path.exists(offload_dir):
    os.makedirs(offload_dir)

# Initialize Accelerator for managing device placement
accelerator = Accelerator()

# Enable 4-bit quantization to reduce memory usage
from transformers import BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_quant_type="nf4",  # Type of quantization
    bnb_4bit_compute_dtype=torch.float16,  # Compute in FP16 for further memory reduction
)

# Load the model and tokenizer from Hugging Face hub with quantization
model_name = "rizal003/vicuna7b-finetuned-liputan6"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the model with the quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    torch_dtype=torch.float16,  # Use FP16 to further reduce memory usage
)

# Move model to accelerator's device (GPU or CPU)
model = accelerator.prepare(model)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['pad_token_id']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from tqdm import tqdm
from evaluate import load as load_metric
import re

# === Load ROUGE metric ===
rouge = load_metric("rouge")

# === Summarize function ===
def summarize_text(text, max_new_tokens=150):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=2048, padding=True).to(model.device)
    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        pad_token_id=tokenizer.eos_token_id
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # Extract only the "Ringkasan :" part if present
    if "Ringkasan :" in decoded:
        summary = decoded.split("Ringkasan :", 1)[1].strip()
    else:
        summary = decoded  # fallback if format isn't found

    # Optional: remove common trailing sources like "( OMI/Putu Setiawan )"
    summary = re.sub(r'\(\s?[A-Z]+\/[A-Za-z\s]+\s?\)', '', summary).strip()

    return summary

# === Run evaluation ===
predictions = []
references = []

print("🚀 Running evaluation without prompt formatting...")
for example in tqdm(eval_dataset, desc="Evaluating"):
    text = example["text"]
    reference = example["summary"]

    prediction = summarize_text(text)
    predictions.append(prediction)
    references.append(reference)

# === Compute ROUGE scores ===
results = rouge.compute(predictions=predictions, references=references)

# === Display the results ===
print("\n📊 ROUGE Evaluation Results:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

🚀 Running evaluation without prompt formatting...


Evaluating: 100%|██████████| 100/100 [14:49<00:00,  8.90s/it]



📊 ROUGE Evaluation Results:
rouge1: 0.1887
rouge2: 0.1284
rougeL: 0.1624
rougeLsum: 0.1664


In [ ]:
# Print an example result
print("\n=== Sample Summary ===")
# print("Original Text:\n", eval_dataset[0]["prompt"])
print("\nReference Summary:\n", references[0])
print("\nModel Prediction:\n", predictions[0])


=== Sample Summary ===

Reference Summary:
 Brasil tidak memberi angin buat Chile . Meski sudah memastikan diri lolos ke putaran final PD 2010 , Brasil tetap tampil solid untuk menundukkan Chile 4-2 ( 2-1 ) di Salvador , Kamis pagi WIB .

Model Prediction:
 Buat ringkasan dari teks berikut:
Liputan6 . com , Salvador : Meskipun sudah memastikan diri melaju ke Afrika Selatan untuk musim panas tahun depan , Brasil tetap bersungguh-sungguh ketika menjamu Chile dalam lanjutan kualifikasi Piala Dunia di Salvador , Rabu ( 9/9 ) atau Kamis dini hari WIB . Ditandai hat-trick Nilmar , Selecao menang 4-2 ( 2-1 ) . Brasil turun tanpa Robinho , Kaka dan Luis Fabiano falam pertandingan yang diwarnai turunnya hujan lebat . Akan tetapi Nilmar mampu menjawab keraguan banyak orang bahwa tuan rumah akan memberi partai ini untuk Chile dengan tujuan menyingkirkan Argentina yang beberapa saat sebelumnya kalah dari Paraguay . Namun Brasil tetap bersungguh-sungguh mempertahankan kehormatan . Nilmar yang mnus

### Testing for E-commerce Review

In [ ]:
!pip install -U datasets fsspec huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.3/489.3 kB 38.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.31.2
    Uninstalling huggingface-hub-0.31.2:
      Successfully uninstalled huggingface-hub-0.31.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.

In [ ]:
# Import library yang diperlukan
import re
import pandas as pd
import nltk
nltk.download("stopwords")
import torch
from datasets import load_dataset
from nltk.corpus import stopwords
from huggingface_hub import login

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
stop_words = set(stopwords.words("indonesian"))

# Load and clean slang
def load_slang_dictionary():
    slang_dataset = load_dataset("theonlydo/indonesia-slang")['train']
    slang_map = {row["slang"]: row["formal"] for row in slang_dataset}
    slang_patterns = {
        re.compile(rf'\b{re.escape(slang)}\b', re.IGNORECASE): formal
        for slang, formal in slang_map.items()
    }
    return slang_patterns

# Clean + preprocess reviews
def preprocess_text(text, slang_patterns):
    for pattern, formal in slang_patterns.items():
        text = pattern.sub(formal, text)
    words = re.findall(r'\b\w+\b', text.lower())
    cleaned_words = [word for word in words if word not in stop_words]
    return " ".join(cleaned_words)

# Load and preprocess dataset
dataset = load_dataset("dipawidia/ecommerce-product-reviews-sentiment", split="train")
dataset = dataset.remove_columns([col for col in dataset.column_names if col != "review"])
slang_patterns = load_slang_dictionary()
dataset = dataset.map(lambda x: {"review": preprocess_text(x["review"], slang_patterns)})

reviews = [row["review"] for row in dataset.select(range(100))] ### Banyak review yang di ambil (jangan di ubah)
joined_reviews = ". ".join(reviews)

README.md:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

labeledReview.datasetFix.json:   0%|          | 0.00/3.15M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11606 [00:00<?, ? examples/s]

Repo card metadata block was not found. Setting CardData to empty.


slang-indo.csv:   0%|          | 0.00/65.5k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4412 [00:00<?, ? examples/s]

Map:   0%|          | 0/11606 [00:00<?, ? examples/s]

In [ ]:
import re

def clean_reviews(text):
    text = re.sub(r"[^a-zA-Z0-9.,;:()\[\]\n\r\s\-+_~!?@%&=|]", " ", text)
    text = re.sub(r"\s+", " ", text)  # normalize spaces
    text = re.sub(r'(barang|produk|kualitas)\s+\1+', r'\1', text, flags=re.IGNORECASE)  # hapus pengulangan
    return text.strip()

joined_reviews = clean_reviews(joined_reviews)
joined_reviews

'suara mic puas pesan nyala pastikan klik nyala nyala. good mudah mudahan awet terima kasih. kondisi dus robek2 tolong perhatikan. kualitas produk harga terjangkau pengirimannya cepat banget 3x order oke sayangnya kalo kado nitip pakai kartu ucapan dibungkus pakai kertas kado. overall bagus tombol berfungsi packaging penyot robek. produk original kualitasnya kecepatan pengiriman sanagt cepat recommended seller. kualitas jelek bocor. terimakasih kak ya barang diterima sukses kak. body nya berat 1579 spiral nya tipis dibanding 1579 semoga awet beli 1659 penasaran 1579 rusak mesin nya harga murah 4k 1579 fast delivery courier ramah ty seller. pesanan terima tanggal 27 january 2021 buka pagi tanggal 29 january 2021 06 00 tab sketsa advan nyalakan cas posisi tab menyala. sesuai gambar. ditingkatkan packagingnya masak iya pakai clear wrap dimasuki kantong plastik wrap sok far keluhan produknya harga bersaing. barang bagus cek ya baut hilang 1 cari bengkel pengiriman kurir cepat banget. baran

In [ ]:
from textwrap import wrap
import re
import torch

# === Configuration ===
max_model_input_length = 2048
max_new_tokens = 200
chunk_char_size = 1500

# Prompt templates
chunk_instruction_template = "[INST] Buat ringkasan dari teks berikut ke dalam satu paragraf:\n{}\n[/INST]"
final_instruction_template = "[INST] Buat ringkasan dari teks berikut ke dalam satu paragraf:\n{}\n[/INST]"

# === Function to clean output ===
def clean_summary(text):
    # Remove fragments after known conjunctions or keywords
    text = re.sub(r'(?<!\w)(?:dan|atau|jika|karena|tetapi|seller)\s*$', '', text.strip(), flags=re.IGNORECASE)
    # Trim to last full sentence
    if '.' in text:
        text = '.'.join(text.split('.')[:-1]) + '.'
    return text.strip()

# === Summarize a single chunk or full content ===
def summarize_text(text, use_final_prompt=False):
    prompt = (
        final_instruction_template.format(text)
        if use_final_prompt else
        chunk_instruction_template.format(text)
    )

    final_token_limit = 400 if use_final_prompt else max_new_tokens

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=max_model_input_length,
        padding=True
    ).to(model.device)

    outputs = model.generate(
        inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=final_token_limit,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_beams=4,
        repetition_penalty=1.2,
        early_stopping=True,
        do_sample=False
    )

    raw_output = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    if "[/INST]" in raw_output:
        summary = raw_output.split("[/INST]")[-1].strip()
    else:
        summary = raw_output

    return clean_summary(summary)

# === Recursive summarization ===
def recursive_summarize(text, max_recursions=5):
    current_text = text
    recursion = 0

    while True:
        recursion += 1
        print(f"\n⏳ Summarization Round #{recursion}...")

        chunks = wrap(current_text, chunk_char_size)
        chunk_summaries = [summarize_text(chunk) for chunk in chunks]
        combined_summary = "\n".join(chunk_summaries)

        if len(combined_summary) <= chunk_char_size or recursion >= max_recursions:
            return summarize_text(combined_summary, use_final_prompt=True)

        current_text = combined_summary

# === Final concise summary ===
# `joined_reviews` should be a long string of reviews or input text
final_summary = recursive_summarize(joined_reviews)
print("\n📌 FINAL SUMMARY:")
print(final_summary)


⏳ Summarization Round #1...







📌 FINAL SUMMARY:



In [ ]:
# Longer review input
input_text = """
[INST] Ringkas ulasan berikut menjadi satu paragraf singkat tanpa mengulang kata-kata secara utuh:\n
Buat ringkasan dari teks berikut ke dalam satu paragraf:
Saya membeli produk ini karena banyak review positif yang saya lihat sebelumnya. Setelah barang sampai, saya langsung coba dan ternyata kualitasnya sangat memuaskan. Materialnya terasa kokoh dan sesuai deskripsi. Pengiriman juga sangat cepat, hanya butuh dua hari untuk sampai di rumah saya di Jakarta. Seller sangat komunikatif dan membantu menjawab semua pertanyaan saya sebelum membeli. Selain itu, kemasan produk sangat rapi dan aman, tidak ada kerusakan sedikit pun. Harganya juga cukup bersaing dibanding toko lain. Saya sudah coba pakai selama seminggu dan sejauh ini tidak ada kendala. Benar-benar pengalaman belanja yang memuaskan dan saya sangat merekomendasikan produk ini kepada orang lain.
[/INST]
"""

# Tokenize input
inputs = tokenizer(
    input_text,
    return_tensors="pt",
    truncation=True,
    max_length=2048,
).to(model.device)

# Generate summary
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        num_beams=4,
        repetition_penalty=1.2,
        do_sample=False
    )

# Decode and print result
summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
print("📌 Model Output:\n", summary)

📌 Model Output:
 
[INST] Ringkas ulasan berikut menjadi satu paragraf singkat tanpa mengulang kata-kata secara utuh:

Buat ringkasan dari teks berikut ke dalam satu paragraf:
Saya membeli produk ini karena banyak review positif yang saya lihat sebelumnya. Setelah barang sampai, saya langsung coba dan ternyata kualitasnya sangat memuaskan. Materialnya terasa kokoh dan sesuai deskripsi. Pengiriman juga sangat cepat, hanya butuh dua hari untuk sampai di rumah saya di Jakarta. Seller sangat komunikatif dan membantu menjawab semua pertanyaan saya sebelum membeli. Selain itu, kemasan produk sangat rapi dan aman, tidak ada kerusakan sedikit pun. Harganya juga cukup bersaing dibanding toko lain. Saya sudah coba pakai selama seminggu dan sejauh ini tidak ada kendala. Benar-benar pengalaman belanja yang memuaskan dan saya sangat merekomendasikan produk ini kepada orang lain. 
[/INST]

Ringkasan dari teks berikut adalah: "Saya membeli produk ini karena banyak review positif yang saya lihat sebelu